<a href="https://colab.research.google.com/github/XiaoLiang28/Datamanagement/blob/main/ps3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up

In [10]:
#---------SETUP----------------------------------
#get useful libraries
import time, os, sys, re #basics
import zipfile, json, datetime, string   #string for annotating points in scatter
import numpy as np #basic math
from statistics import * #stats

import matplotlib.pyplot as plt #import pylab as plt #apparently discouraged now:
 #https://stackoverflow.com/questions/11469336/what-is-the-difference-between-pylab-and-pyplot
 #https://www.tutorialspoint.com/matplotlib/matplotlib_pylab_module.htm

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
#s4 = Styler(df4, uuid_len=0, cell_ids=False)

import urllib  #weird, guess need to have os and pandas imported for this to work  %TODO/LATER ditch it, its weird anyway, just use wget/curl

from google.colab import files

#import webbrowser

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter() #this enables spreadsheet view upon calling dataframe (without() )

#many tricks how to extend notebook functionality
#https://coderzcolumn.com/tutorials/python/list-of-useful-magic-commands-in-jupyter-notebook-lab
#will display all output not just last command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#MAGICS and THEMES/STYLES: important! does affect not just shading/colors, but also fonts, spacing, etc
#(even if you only select default (v not selecting anything) [but does seem to work better if you do make explicit sleections])

###magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html
#most essential setup for vis: it does affect vis! careful!! stick with inline, maybe notebook; others mostly for non-notebook, eg spyder environ
#https://jakevdp.github.io/PythonDataScienceHandbook/04.00-introduction-to-matplotlib.html recomends *inline*!
#show current one:
#%matplotlib
#%matplotlib --list
#interactive plots:
#%matplotlib notebook
#static images of your plot:
%matplotlib inline
#may play with this one and other magics (btw default is probably agg)
#%matplotlib nbagg

###themes/styles: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
#https://jakevdp.github.io/PythonDataScienceHandbook/04.11-settings-and-stylesheets.html
#https://matplotlib.org/stable/tutorials/introductory/customizing.html
#here more about art and style than under the hood functionality as with magics, explore and experiment
#many may find 'default' or seaborn ones more pleasing; my fav 'classic' is back from 90s ;)
#plt.style.available #list available styles :) may install more
#plt.style.use('default') # more delicate subtle than classic
plt.style.use('classic')  #  'seaborn-whitegrid' 'seaborn-white' 'seaborn-poster'
# btw: magics v theme/style sequence matters, eg if i specify classic style before inline magic, i wouldnt get grey bounding box im getting

# Project 1

## Data Import

### Human Development Index (HDI)

In [3]:
hdi=pd.read_excel("https://github.com/XiaoLiang28/Datamanagement/raw/main/Human%20Development%20Index(1)%20(1).xlsx")

In [4]:
hdi

,HDI Rank,Country,Human Development Index (HDI),Unnamed: 3,Life expectancy at birth,Unnamed: 5,Expected years of schooling,Unnamed: 7,Mean years of schooling,Unnamed: 9,Gross national income (GNI) per capita,Unnamed: 11,Human Development
0,1,Switzerland,0.962,NaN,83.9872,NaN,16.500299,NaN,13.859660,NaN,66933.004540,NaN,Very High
1,2,Norway,0.961,NaN,83.2339,NaN,18.185200,NaN,13.003630,NaN,64660.106220,NaN,Very High
2,3,Iceland,0.959,NaN,82.6782,NaN,19.163059,NaN,13.767170,NaN,55782.049810,NaN,Very High
3,4,"Hong Kong, China (SAR)",0.952,NaN,85.4734,NaN,17.278170,NaN,12.226210,NaN,62606.845400,NaN,Very High
4,5,Australia,0.951,NaN,84.5265,NaN,21.054590,NaN,12.726820,NaN,49238.433350,NaN,Very High
...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,187,Burundi,0.426,NaN,61.6627,NaN,10.722722,NaN,3.129267,NaN,731.786709,NaN,Low
187,188,Central African Republic,0.404,NaN,53.8947,NaN,8.040172,NaN,4.334000,NaN,966.058611,NaN,Low
188,189,Niger,0.400,NaN,61.5763,NaN,6.957112,NaN,2.116717,NaN,1239.866936,NaN,Low
189,190,Chad,0.394,NaN,52.5254,NaN,8.035914,NaN,2.573774,NaN,1364.169417,NaN,Low


In [5]:
hdi = hdi.drop(['Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7', 'Unnamed: 9', 'Unnamed: 11'], axis=1)

In [6]:
hdi

,HDI Rank,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,Human Development
0,1,Switzerland,0.962,83.9872,16.500299,13.859660,66933.004540,Very High
1,2,Norway,0.961,83.2339,18.185200,13.003630,64660.106220,Very High
2,3,Iceland,0.959,82.6782,19.163059,13.767170,55782.049810,Very High
3,4,"Hong Kong, China (SAR)",0.952,85.4734,17.278170,12.226210,62606.845400,Very High
4,5,Australia,0.951,84.5265,21.054590,12.726820,49238.433350,Very High
...,...,...,...,...,...,...,...,...
186,187,Burundi,0.426,61.6627,10.722722,3.129267,731.786709,Low
187,188,Central African Republic,0.404,53.8947,8.040172,4.334000,966.058611,Low
188,189,Niger,0.400,61.5763,6.957112,2.116717,1239.866936,Low
189,190,Chad,0.394,52.5254,8.035914,2.573774,1364.169417,Low


In [36]:
hdi.dtypes

HDI Rank                                    int64
Country                                    object
Human Development Index (HDI)             float64
Life expectancy at birth                  float64
Expected years of schooling               float64
Mean years of schooling                   float64
Gross national income (GNI) per capita    float64
Human Development                          object
dtype: object

### Gender Inequality Index (GII)

In [7]:
gii=pd.read_excel("https://github.com/XiaoLiang28/Datamanagement/raw/main/Gender%20Inequality%20Index(1)%20(1).xlsx")

In [8]:
gii

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,SDG3.1,NaN,SDG3.7,NaN,...,NaN,SDG4.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Gender Inequality Index,NaN,NaN,NaN,Maternal mortality ratio,NaN,Adolescent birth rate,NaN,...,NaN,Population with at least some secondary education,NaN,NaN,NaN,Labour force participation rate,NaN,NaN,NaN,NaN
3,NaN,NaN,Value,NaN,Rank,NaN,"(deaths per 100,000 live births)",NaN,"(births per 1,000 women ages 15–19)",NaN,...,NaN,(% ages 25 and older),NaN,NaN,NaN,(% ages 15 and older),NaN,NaN,NaN,NaN
4,HDI rank,Country,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Female,NaN,Male,NaN,Female,NaN,Male,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,187,Burundi,0.505,NaN,127,NaN,548,NaN,53.621,NaN,...,NaN,7.79991,NaN,13.046886,NaN,78.991,NaN,77.379,NaN,Low
193,188,Central African Republic,0.672,NaN,166,NaN,829,NaN,160.507,NaN,...,NaN,13.94,NaN,31.56,NaN,63.312,NaN,79.486,NaN,Low
194,189,Niger,0.611,NaN,153,NaN,509,NaN,170.462,NaN,...,NaN,9.163242,NaN,15.232771,NaN,61.73,NaN,84.267,NaN,Low
195,190,Chad,0.652,NaN,165,NaN,1140,NaN,138.306,NaN,...,NaN,7.739064,NaN,24.369996,NaN,46.914,NaN,69.927,NaN,Low


In [11]:
pd.set_option('display.max_columns', None)

In [13]:
pd.set_option('display.max_rows', None)

In [14]:
gii

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,SDG3.1,NaN,SDG3.7,NaN,SDG5.5,NaN,SDG4.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Gender Inequality Index,NaN,NaN,NaN,Maternal mortality ratio,NaN,Adolescent birth rate,NaN,Share of seats in parliament,NaN,Population with at least some secondary education,NaN,NaN,NaN,Labour force participation rate,NaN,NaN,NaN,NaN
3,NaN,NaN,Value,NaN,Rank,NaN,"(deaths per 100,000 live births)",NaN,"(births per 1,000 women ages 15–19)",NaN,(% held by women),NaN,(% ages 25 and older),NaN,NaN,NaN,(% ages 15 and older),NaN,NaN,NaN,NaN
4,HDI rank,Country,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,NaN,Male,NaN,Female,NaN,Male,NaN,NaN
5,NaN,NaN,2021,NaN,2021,NaN,2017,NaN,2021,NaN,2021,NaN,2021,NaN,2021,NaN,2021,NaN,2021,NaN,NaN
6,1,Switzerland,0.018,NaN,3,NaN,5,NaN,2.212,NaN,39.837398,NaN,96.858673,NaN,97.533417,NaN,61.728,NaN,72.705,NaN,Very High
7,2,Norway,0.016,NaN,2,NaN,2,NaN,2.336,NaN,44.970414,NaN,99.094223,NaN,99.283562,NaN,60.294,NaN,71.95,NaN,Very High
8,3,Iceland,0.043,NaN,8,NaN,4,NaN,5.395,NaN,47.619048,NaN,99.768417,NaN,99.662552,NaN,61.671,NaN,70.463,NaN,Very High
9,4,"Hong Kong, China (SAR)",..,NaN,..,NaN,..,NaN,1.584,NaN,..,NaN,77.100433,NaN,83.368141,NaN,53.524,NaN,65.771,NaN,Very High


In [15]:
gii=gii[6:197]

In [16]:
gii

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
6,1,Switzerland,0.018,NaN,3,NaN,5,NaN,2.212,NaN,39.837398,NaN,96.858673,NaN,97.533417,NaN,61.728,NaN,72.705,NaN,Very High
7,2,Norway,0.016,NaN,2,NaN,2,NaN,2.336,NaN,44.970414,NaN,99.094223,NaN,99.283562,NaN,60.294,NaN,71.95,NaN,Very High
8,3,Iceland,0.043,NaN,8,NaN,4,NaN,5.395,NaN,47.619048,NaN,99.768417,NaN,99.662552,NaN,61.671,NaN,70.463,NaN,Very High
9,4,"Hong Kong, China (SAR)",..,NaN,..,NaN,..,NaN,1.584,NaN,..,NaN,77.100433,NaN,83.368141,NaN,53.524,NaN,65.771,NaN,Very High
10,5,Australia,0.073,NaN,19,NaN,6,NaN,8.096,NaN,37.885463,NaN,94.580063,NaN,94.430069,NaN,61.057,NaN,70.547,NaN,Very High
11,6,Denmark,0.013,NaN,1,NaN,4,NaN,1.908,NaN,39.664804,NaN,95.067261,NaN,95.238098,NaN,57.721,NaN,66.653,NaN,Very High
12,7,Sweden,0.023,NaN,4,NaN,4,NaN,3.345,NaN,46.991404,NaN,91.808403,NaN,92.224701,NaN,61.651,NaN,68.009,NaN,Very High
13,8,Ireland,0.074,NaN,21,NaN,5,NaN,5.941,NaN,27.272727,NaN,88.127581,NaN,85.956813,NaN,56.526,NaN,68.573,NaN,Very High
14,9,Germany,0.073,NaN,19,NaN,7,NaN,7.465,NaN,34.782609,NaN,96.07322,NaN,96.462144,NaN,56.843,NaN,66.041,NaN,Very High
15,10,Netherlands,0.025,NaN,5,NaN,5,NaN,2.804,NaN,39.111111,NaN,89.808189,NaN,92.674728,NaN,62.444,NaN,71.315,NaN,Very High


In [17]:
gii.rename(columns={'Unnamed: 0':'HDI rank'},inplace=True)
gii.rename(columns={'Unnamed: 1':'Country'},inplace=True)
gii.rename(columns={'Unnamed: 2':'GII Value'},inplace=True)
gii.rename(columns={'Unnamed: 6':'Maternal mortality ratio'},inplace=True)
gii.rename(columns={'Unnamed: 8':'Adolescent birth rate'},inplace=True)
gii.rename(columns={'Unnamed: 10':'Share of seats in parliament'},inplace=True)
gii.rename(columns={'Unnamed: 12':'Population with at least some secondary education female'},inplace=True)
gii.rename(columns={'Unnamed: 14':'Population with at least some secondary education male'},inplace=True)
gii.rename(columns={'Unnamed: 16':'Labour force participation rate female'},inplace=True)
gii.rename(columns={'Unnamed: 18':'Labour force participation rate male'},inplace=True)
gii.rename(columns={'Unnamed: 20':'Human Development'},inplace=True)

<ipython-input-17-1a19cbcfd6d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gii.rename(columns={'Unnamed: 0':'HDI rank'},inplace=True)
<ipython-input-17-1a19cbcfd6d9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gii.rename(columns={'Unnamed: 1':'Country'},inplace=True)
<ipython-input-17-1a19cbcfd6d9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gii.rename(columns={'Unnamed: 2':'GII Value'},inplace=True)
<ipython-input-17-1

In [18]:
gii =gii.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 7', 'Unnamed: 9', 'Unnamed: 11', 'Unnamed: 13', 'Unnamed: 15', 'Unnamed: 17', 'Unnamed: 19'], axis=1)

In [19]:
gii = gii.replace('..', np.nan)

In [20]:
gii

,HDI rank,Country,GII Value,Maternal mortality ratio,Adolescent birth rate,Share of seats in parliament,Population with at least some secondary education female,Population with at least some secondary education male,Labour force participation rate female,Labour force participation rate male,Human Development
6,1,Switzerland,0.018,5.0,2.212,39.837398,96.858673,97.533417,61.728,72.705,Very High
7,2,Norway,0.016,2.0,2.336,44.970414,99.094223,99.283562,60.294,71.950,Very High
8,3,Iceland,0.043,4.0,5.395,47.619048,99.768417,99.662552,61.671,70.463,Very High
9,4,"Hong Kong, China (SAR)",NaN,NaN,1.584,NaN,77.100433,83.368141,53.524,65.771,Very High
10,5,Australia,0.073,6.0,8.096,37.885463,94.580063,94.430069,61.057,70.547,Very High
11,6,Denmark,0.013,4.0,1.908,39.664804,95.067261,95.238098,57.721,66.653,Very High
12,7,Sweden,0.023,4.0,3.345,46.991404,91.808403,92.224701,61.651,68.009,Very High
13,8,Ireland,0.074,5.0,5.941,27.272727,88.127581,85.956813,56.526,68.573,Very High
14,9,Germany,0.073,7.0,7.465,34.782609,96.073220,96.462144,56.843,66.041,Very High
15,10,Netherlands,0.025,5.0,2.804,39.111111,89.808189,92.674728,62.444,71.315,Very High


### Gender Development Index (GDI)

In [23]:
gdi=pd.read_excel("https://github.com/XiaoLiang28/Datamanagement/raw/main/GDI.xlsx")

In [24]:
gdi

,Unnamed: 0,Country,GDI
0,NaN,Afghanistan,0.681
1,NaN,Albania,1.010
2,NaN,Algeria,0.880
3,NaN,Angola,0.903
4,NaN,Argentina,1.000
5,NaN,Armenia,1.000
6,NaN,Australia,0.970
7,NaN,Austria,0.980
8,NaN,Azerbaijan,0.970
9,NaN,Bahrain,0.930


In [25]:
gdi.drop(['Unnamed: 0'],axis=1, inplace=True)

In [26]:
gdi

,Country,GDI
0,Afghanistan,0.681
1,Albania,1.010
2,Algeria,0.880
3,Angola,0.903
4,Argentina,1.000
5,Armenia,1.000
6,Australia,0.970
7,Austria,0.980
8,Azerbaijan,0.970
9,Bahrain,0.930


### World Happiness Report

In [29]:
happiness=pd.read_excel("https://github.com/XiaoLiang28/Datamanagement/raw/main/Happiness%202021.xls")

In [30]:
happiness

,Rank,Country,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,1,Finland,Western Europe,7.8421,0.031646,7.904126,7.780075,10.775202,0.953603,72.000000,0.949268,-0.097760,0.185846,2.430343,1.446356,1.106192,0.740824,0.690577,0.124200,0.480524,3.253415
1,2,Denmark,Western Europe,7.6195,0.034657,7.687428,7.551572,10.933176,0.954410,72.699753,0.945639,0.030109,0.178838,2.430343,1.501548,1.108008,0.762862,0.686160,0.207615,0.484993,2.868296
2,3,Switzerland,Western Europe,7.5715,0.036243,7.642536,7.500463,11.117368,0.941742,74.400101,0.918788,0.024629,0.291698,2.430343,1.565899,1.079469,0.816415,0.653488,0.204040,0.413019,2.839148
3,4,Iceland,Western Europe,7.5539,0.059373,7.670271,7.437528,10.877768,0.982938,73.000000,0.955123,0.160274,0.672865,2.430343,1.482190,1.172279,0.772319,0.697701,0.292526,0.169940,2.966945
4,5,Netherlands,Western Europe,7.4640,0.027326,7.517559,7.410441,10.931812,0.941601,72.400116,0.913116,0.175404,0.337938,2.430343,1.501072,1.079151,0.753425,0.646585,0.302397,0.383531,2.797818
5,6,Norway,Western Europe,7.3925,0.035449,7.461979,7.323021,11.053126,0.954445,73.300011,0.959588,0.093426,0.270028,2.430343,1.543455,1.108088,0.781768,0.703135,0.248919,0.426839,2.580301
6,7,Sweden,Western Europe,7.3627,0.035700,7.432671,7.292729,10.866995,0.933805,72.700050,0.944604,0.086223,0.237460,2.430343,1.478426,1.061588,0.762872,0.684902,0.244220,0.447608,2.683093
7,8,Luxembourg,Western Europe,7.3244,0.036685,7.396303,7.252497,11.646564,0.907790,72.599998,0.907180,-0.033566,0.386419,2.430343,1.750786,1.002978,0.759721,0.639363,0.166077,0.352614,2.652866
8,9,New Zealand,North America and ANZ,7.2766,0.040186,7.355364,7.197836,10.642713,0.948148,73.400146,0.928580,0.134256,0.241641,2.430343,1.400068,1.093900,0.784921,0.665403,0.275554,0.444942,2.611801
9,10,Austria,Western Europe,7.2678,0.035523,7.337424,7.198175,10.906316,0.934176,73.299721,0.907691,0.041568,0.481378,2.430343,1.492164,1.062423,0.781758,0.639984,0.215090,0.292056,2.784285


## Merge

In [38]:
hdi_gii = pd.merge(hdi, gii, how='outer', on=['Country'],indicator=True)
hdi_gii

KeyError: ignored